In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import argparse
from functools import partial
from joblib import Parallel, delayed
from datetime import datetime
import os
from pathlib import Path
import torch
from scipy.stats import norm

from folktables import ACSDataSource, ACSPublicCoverage
from folktables import BasicProblem

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json

from data import *

In [3]:
def get_confusion_matrix(Yhat, Y):
    n = Y.shape[0]
    P = (Y==True).sum()  # positives
    PP = (Yhat==True).sum()  # positive predictions
    TP = ((Yhat==True) & (Y==True)).sum()  # true positives
    TN = ((Yhat==False) & (Y==False)).sum()  # true negatives
    M_P = P/n
    M_PP = PP/n
    M_TPR = TP/P
    M_TNR = TN/(n-P)
    M_PPV = TP/PP
    M_NPV = TN/(n-PP)
    var_PP = M_PP * (1-M_PP)
    var_TPR = 1/M_P * M_TPR*(1-M_TPR)
    var_TNR = 1/(1-M_P) * M_TNR*(1-M_TNR)
    var_PPV = 1/M_PP * M_TPR*(1-M_TPR)
    var_NPV = 1/(1-M_PP) * M_TNR*(1-M_TNR)
    return {'M_P': M_P, 'M_PP': M_PP, 'M_TPR': M_TPR, 'M_TNR': M_TNR, 'M_PPV': M_PPV, 'M_NPV': M_NPV,
           'var_PP': var_PP, 'var_TPR': var_TPR, 'var_TNR': var_TNR, 'var_PPV': var_PPV, 'var_NPV': var_NPV}

# Preparing ACS data

In [4]:
ACS_RACE_CODE = {
    'White': 1,
    'BlackorAA': 2,
    'Asian': 6,
}

In [5]:
us_state = 'CA'
outcome = 'income_binary'
group = 'BlackorAA'

data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person',
                              root_dir='data')
data = data_source.get_data(states=['CA'], download=True)

In [6]:
ACSQuery = create_acs_query_object_model(outcome)
X_acs, Y_acs, G_acs = ACSQuery.df_to_pandas(data)
Y_acs = Y_acs.to_numpy().squeeze()
G_acs = G_acs.to_numpy().squeeze()

In [7]:
# Remove rows with nan target value
nan_indices = np.isnan(Y_acs)
X_acs = X_acs[~nan_indices]
Y_acs = Y_acs[~nan_indices]
G_acs = G_acs[~nan_indices]

Y_acs = Y_acs.astype(bool)
print("Counts and Y mean per group", [(i, np.sum(G_acs==i), Y_acs[G_acs==i].mean()) for i in np.unique(G_acs)])

Counts and Y mean per group [(1, 121006, 0.4433664446391088), (2, 8557, 0.3448638541544934), (3, 1294, 0.2836166924265842), (4, 13, 0.38461538461538464), (5, 450, 0.21333333333333335), (6, 32709, 0.4805099513895258), (7, 637, 0.29827315541601257), (8, 22793, 0.19427017066643268), (9, 8206, 0.3571776748720448)]


In [8]:
group_code = ACS_RACE_CODE[group]
subset_by_race = np.isin(G_acs, [1,group_code])
X_acs = X_acs[subset_by_race]
G_acs = G_acs[subset_by_race]
Y_acs = Y_acs[subset_by_race]
G_acs = (G_acs==group_code).astype(int)
X_acs = X_acs.drop(['PWGTP','FHISP'], axis=1)

In [9]:
X_train, X_test, Y_train, Y_test, G_train, G_test = train_test_split(
        X_acs, Y_acs, G_acs, test_size=0.3, random_state=0)

In [10]:
print('n %d n1 %d G %0.2f' % (X_test.shape[0], G_test.sum(), G_test.mean()))

n 38869 n1 2562 G 0.07


# Train classification model

In [11]:
model = make_pipeline(StandardScaler(), GradientBoostingClassifier())
model.fit(X_train, Y_train)
Yhat = model.predict(X_test)

In [12]:
G1_idx = (G_test==1)
G1_matrix = get_confusion_matrix(Yhat[G1_idx], Y_test[G1_idx])
G0_matrix = get_confusion_matrix(Yhat[~G1_idx], Y_test[~G1_idx])

In [13]:
print(G1_matrix['var_PP'],G1_matrix['var_TPR'],G1_matrix['var_TNR'],G1_matrix['var_PPV'],G1_matrix['var_NPV'])
print(G0_matrix['var_PP'],G0_matrix['var_TPR'],G0_matrix['var_TNR'],G0_matrix['var_PPV'],G0_matrix['var_NPV'])

0.22077261434001172 0.6220315517099292 0.18463855866324966 0.6574497183553345 0.17948285719970342
0.24687845720238097 0.37752457961498664 0.25498979736120814 0.3743639087158845 0.25669545699076723


In [14]:
U_DP = G1_matrix['M_P'] - G0_matrix['M_P']
U_TPR = G1_matrix['M_TPR'] - G0_matrix['M_TPR']
U_TNR = G1_matrix['M_TNR'] - G0_matrix['M_TNR']
U_PPV = G1_matrix['M_PPV'] - G0_matrix['M_PPV']
U_NPV = G1_matrix['M_NPV'] - G0_matrix['M_NPV']

var_U_DP = (np.sqrt(G1_matrix['var_PP']) + np.sqrt(G0_matrix['var_PP']))**2
var_U_TPR = (np.sqrt(G1_matrix['var_TPR']) + np.sqrt(G0_matrix['var_TPR']))**2
var_U_TNR = (np.sqrt(G1_matrix['var_TNR']) + np.sqrt(G0_matrix['var_TNR']))**2
var_U_PPV = (np.sqrt(G1_matrix['var_PPV']) + np.sqrt(G0_matrix['var_PPV']))**2
var_U_NPV = (np.sqrt(G1_matrix['var_NPV']) + np.sqrt(G0_matrix['var_NPV']))**2

print('metric DP %f, TPR %f, TNR %f, PPV %f, NPV %f' % (U_DP,U_TPR,U_TNR,U_PPV,U_NPV))
print('variance DP %f, TPR %f, TNR %f, PPV %f, NPV %f' % (var_U_DP,var_U_TPR,var_U_TNR,var_U_PPV,var_U_NPV))

U_DP,U_TPR,U_TNR,U_PPV,U_NPV = np.abs(U_DP),np.abs(U_TPR),np.abs(U_TNR),np.abs(U_PPV),np.abs(U_NPV)

metric DP -0.092636, TPR -0.105867, TNR 0.032381, PPV -0.060340, NPV 0.002833
variance DP 0.934573, TPR 1.968746, TNR 0.873591, PPV 2.024034, NPV 0.865468


In [15]:
print('DP G1 %0.04f, G0 %0.04f' % (G1_matrix['M_P'],G0_matrix['M_P']))
print('DP var G1 %0.04f, G0 %0.04f' % (G1_matrix['M_P']*(1-G1_matrix['M_P']),G0_matrix['M_P']*(1-G0_matrix['M_P'])))

DP G1 0.3478, G0 0.4404
DP var G1 0.2268, G0 0.2464


In [16]:
(np.sqrt(0.227)+np.sqrt(0.246))**2

0.9456182391740716

In [17]:
alpha=0.05
beta=0.2
(norm.ppf(1-alpha/2)+norm.ppf(1-beta))**2 * 0.946 / 0.093**2

858.4854004733769

In [18]:
print('TPR G1 %0.2f, G0 %0.2f' % (G1_matrix['M_TPR'],G0_matrix['M_TPR']))

TPR G1 0.68, G0 0.79


In [19]:
print('Accuracy %0.2f, G1 %0.2f, G0 %0.2f' % ((Yhat==Y_test).mean(),(Yhat[G1_idx]==Y_test[G1_idx]).mean(),(Yhat[~G1_idx]==Y_test[~G1_idx]).mean()))

Accuracy 0.81, G1 0.80, G0 0.81


# Get sample sizes

In [20]:
def get_sample_size(alpha, beta, var, Utol, tau):
    return (norm.ppf(1-alpha/2)+norm.ppf(1-beta))**2 * var / (tau-Utol)**2

In [21]:
alpha = 0.05
beta = 0.2
Utol = 0
tau_fraction = 1

n_DP = get_sample_size(alpha, beta, var=var_U_DP, Utol=Utol, tau=tau_fraction*U_DP)
n_TPR = get_sample_size(alpha, beta, var=var_U_TPR, Utol=Utol, tau=tau_fraction*U_TPR)
n_TNR = get_sample_size(alpha, beta, var=var_U_TNR, Utol=Utol, tau=tau_fraction*U_TNR)
n_PPV = get_sample_size(alpha, beta, var=var_U_PPV, Utol=Utol, tau=tau_fraction*U_PPV)
n_NPV = get_sample_size(alpha, beta, var=var_U_NPV, Utol=Utol, tau=tau_fraction*U_NPV)

print('DP & %0.2f & %0.2f' % (U_DP,n_DP))
print('TPR & %0.2f & %0.2f' % (U_TPR,n_TPR))
print('TNR & %0.2f & %0.2f' % (U_TNR,n_TNR))
print('PPV & %0.2f & %0.2f' % (U_PPV,n_PPV))
print('NPV & %0.3f & %0.2f' % (U_NPV,n_NPV))

DP & 0.09 & 854.80
TPR & 0.11 & 1378.73
TNR & 0.03 & 6539.25
PPV & 0.06 & 4363.33
NPV & 0.003 & 846670.97
